In [5]:
import torch
import torch.nn as nn
from torch.nn import init
import functools
from torch.optim import lr_scheduler
# from util.image_pool import ImagePool
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import torchvision

In [6]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.InstanceNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.InstanceNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


In [7]:
import torch.nn as nn
import torch.nn.init
import torch as torch
import torch . nn . functional as F

class Generator_Model(torch.nn.Module):
    
    def relu(self,x):
        return nn.ReLU(x)
    def sigmoid(self,x):
        return nn.Sigmoid(x)      
    
    def __init__(self):
        
        super ( Generator_Model , self ). __init__ ()

        self.conv1 = torch.nn.Conv2d(1, 64, 7, stride=1, bias=True)
        torch.nn.init.xavier_normal(self.conv1.weight)
        self.bn1 = torch.nn.InstanceNorm2d(64)

        self.conv2 = torch.nn.Conv2d(64, 128, 3, stride=2, padding=0, bias=True)
        torch.nn.init.xavier_normal(self.conv2.weight)
        self.bn2 = torch.nn.InstanceNorm2d(128)

        self.conv3 = torch.nn.Conv2d(128, 256, 3, stride=2, padding=0, bias=True)
        torch.nn.init.xavier_normal(self.conv3.weight)
        self.bn3 = torch.nn.InstanceNorm2d(256)

        self.resnet10 = torch.nn.Sequential(*[BasicBlock(256,256) for x in range(10)])
        
        self.conv3_2 = torch.nn.ConvTranspose2d(256, 128, 3, stride=2, output_padding=1, bias=True)
        torch.nn.init.xavier_normal(self.conv3_2.weight)
        self.bn3_2 = torch.nn.InstanceNorm2d(128)
        
        self.conv2_1 = torch.nn.ConvTranspose2d(128, 64, 3, stride=2, output_padding=1, bias=True)
        torch.nn.init.xavier_normal(self.conv2_1.weight)
        self.bn2_1 = torch.nn.InstanceNorm2d(64)
        
        self.conv1_0 = torch.nn.ConvTranspose2d(64, 1, 7, stride=1, padding=0, bias=True)
        torch.nn.init.xavier_normal(self.conv1_0.weight)
        
        
    def forward(self,data):
        x = F.relu(self.bn1(self.conv1(data)))
#         print(x.shape)
        x = F.relu(self.bn2(self.conv2(x)))
#         print(x.shape)
        x = F.relu(self.bn3(self.conv3(x)))
#         print(x.shape)
        
        x = self.resnet10(x)
#         print(x.shape)
        x = F.relu(self.bn3_2(self.conv3_2(x)))
#         print(x.shape)
        x = F.relu(self.bn2_1(self.conv2_1(x)))
#         print(x.shape)
        x = F.sigmoid(self.conv1_0(x))
#         print(x.shape)
#         print('end')
        return x

In [8]:
class Discriminator_Model(torch.nn.Module):
    
    def lrelu(self,x):
        return nn.LeakyReLU(x)
    def sigmoid(self,x):
        return nn.Sigmoid(x)      
    
    def __init__(self):
        super ( Discriminator_Model , self ). __init__ ()
        self.conv1 = torch.nn.Conv2d(1, 64, 4, stride=2, padding=1, bias=True)
        torch.nn.init.xavier_normal(self.conv1.weight)

        self.conv2 = torch.nn.Conv2d(64, 256, 4, stride=2, padding=1, bias=True)
        torch.nn.init.xavier_normal(self.conv2.weight)
        self.bn2 = torch.nn.InstanceNorm2d(256)

        self.conv3 = torch.nn.Conv2d(256, 1, 1, stride=1, padding=0, bias=True)
        torch.nn.init.xavier_normal(self.conv3.weight)

    def forward(self,data):
#         print(data.shape)
        x = F.leaky_relu(self.conv1(data))
#         print(x.shape)
        x = F.leaky_relu(self.bn2(self.conv2(x)))
#         print(x.shape)
        x = self.conv3(x)
#         print(x.shape)
        return x

In [9]:
class CycleGAN():
    
    def __init__(self):
        self.G_A_to_B = Generator_Model()
        self.G_B_to_A = Generator_Model()
        self.D_A = Discriminator_Model()
        self.D_B = Discriminator_Model()
        self.D_A_m = Discriminator_Model()
        self.D_B_m = Discriminator_Model()
        
        if torch.cuda.is_available():
            self.G_A_to_B = self.G_A_to_B.cuda()
            self.G_B_to_A = self.G_B_to_A.cuda()
            self.D_A = self.D_A.cuda()
            self.D_B = self.D_B.cuda()
            self.D_A_m = self.D_A_m.cuda()
            self.D_B_m = self.D_B_m.cuda()

        
    def loss_A_to_B(self,x_A):
        x_B_hat = self.G_A_to_B(x_A)
        _D_B_hat = self.D_B(x_B_hat)
        
        return torch.norm(_D_B_hat.view(-1,16*21)-1,2,dim=1)
        
        
    def loss_B_to_A(self,x_B):
        x_A_hat = self.G_B_to_A(x_B)
        _D_A_hat = self.D_A(x_A_hat)
        
        return torch.norm(_D_A_hat.view(-1,16*21)-1,2,dim=1)
    
    def loss_C(self,x_A,x_B):
        x_B_hat = self.G_A_to_B(x_A)
        x_A_hat = self.G_B_to_A(x_B)
        x_B_til = self.G_A_to_B(x_A_hat)
        x_A_til = self.G_B_to_A(x_B_hat)        
        
        return torch.norm((x_A_til-x_A).view(-1,64*84),1,dim=1)+torch.norm((x_B_til-x_B).view(-1,64*84),1,dim=1)
    
    def loss_G(self,_lambda,x_A,x_B):
        a = self.loss_A_to_B(x_A)+self.loss_B_to_A(x_B)+self.loss_C(x_A,x_B)*_lambda
        return torch.sum(a)
    
    def loss_DA(self,x_A,x_B):
        _D_A = self.D_A(x_A)
        x_A_hat = self.G_B_to_A(x_B)
        _D_A_hat = self.D_A(x_A_hat.detach())
        return torch.sum(0.5*(torch.norm(_D_A.view(-1,16*21)-1,2,dim=1) + torch.norm(_D_A_hat.view(-1,16*21),2,dim=1)))
    
    def loss_DB(self,x_A,x_B):
        _D_B = self.D_B(x_B)
        x_B_hat = self.G_A_to_B(x_A)
        _D_B_hat = self.D_B(x_B_hat.detach())
        
        return torch.sum(0.5*(torch.norm(_D_B.view(-1,16*21)-1,2,dim=1) + torch.norm(_D_B_hat.view(-1,16*21),2,dim=1)))
    
    def loss_DA_m(self,_gamma,x_A,x_B,x_M):
        x_A_hat = self.G_B_to_A(x_B)
        _D_A_m = self.D_A_m(x_M)
        _D_A_m_hat = self.D_A_m(x_A_hat.detach())
        return torch.sum(0.5*_gamma*(torch.norm(_D_A_m.view(-1,16*21)-1,2,dim=1) + torch.norm(_D_A_m_hat.view(-1,16*21),2,dim=1)))
    
    def loss_DB_m(self,_gamma,x_A,x_B,x_M):
        x_B_hat = self.G_A_to_B(x_A)
        _D_B_m = self.D_B_m(x_M)
        _D_B_m_hat = self.D_B_m(x_B_hat.detach())
        
        return torch.sum(0.5*_gamma*(torch.norm(_D_B_m.view(-1,16*21)-1,2,dim=1) + torch.norm(_D_B_m_hat.view(-1,16*21),2,dim=1)))
    
#     def loss_D(self,_gamma,x_A,x_B,x_M):
#         a = self.loss_DA(x_A,x_B) + self.loss_DB(x_A,x_B) + _gamma*(self.loss_DA_m(x_A,x_B,x_M) + self.loss_DB_m(x_A,x_B,x_M))
#         return torch.sum(a)
    
#     def total_loss(self,_gamma,_lambda):
#         loss = self.loss_D(_gamma)+self.loss_G(_lambda)
#         return torch.sum(loss)


In [10]:
classic_test = np.load('../classic_test_piano.npy').astype(np.float32)
classic_train = np.load('../classic_train_piano.npy').astype(np.float32)
jazz_test = np.load('../jazz_test_piano.npy').astype(np.float32)
jazz_train = np.load('../jazz_train_piano.npy').astype(np.float32)
pop_test = np.load('../pop_test_piano.npy').astype(np.float32)
pop_train = np.load('../pop_train_piano.npy').astype(np.float32)

In [11]:
if not torch.cuda.is_available():
# if True:
    classic_test_tensor = torch.from_numpy(classic_test)
    classic_train_tensor = torch.from_numpy(classic_train)
    jazz_test_tensor = torch.from_numpy(jazz_test)
    jazz_train_tensor = torch.from_numpy(jazz_train)
    pop_test_tensor = torch.from_numpy(pop_test)
    pop_train_tensor = torch.from_numpy(pop_train)

else:
    classic_test_tensor = torch.from_numpy(classic_test).cuda()
    classic_train_tensor = torch.from_numpy(classic_train).cuda()
    jazz_test_tensor = torch.from_numpy(jazz_test).cuda()
    jazz_train_tensor = torch.from_numpy(jazz_train).cuda()
    pop_test_tensor = torch.from_numpy(pop_test).cuda()
    pop_train_tensor = torch.from_numpy(pop_train).cuda()
    
classic_test_var = Variable(classic_test_tensor.permute(0,3,1,2))
classic_train_var = Variable(classic_train_tensor.permute(0,3,1,2))
jazz_test_var = Variable(jazz_test_tensor.permute(0,3,1,2))
jazz_train_var = Variable(jazz_train_tensor.permute(0,3,1,2))    
pop_test_var = Variable(pop_test_tensor.permute(0,3,1,2))
pop_train_var = Variable(pop_train_tensor.permute(0,3,1,2))   
    

In [12]:
import itertools
x_A = classic_train_var
x_B = pop_train_var
x_M = torch.cat((x_A,x_B),0)
n_M = np.random.permutation(x_M.shape[0])
x_M = x_M[n_M]



lr = .0002
epochs = 30
B = 8
N = min(x_A.shape[0],x_B.shape[0])
NB = (N + B - 1) / B
cycleGAN = CycleGAN()


optimizerG = torch . optim . Adam ( itertools.chain(cycleGAN.G_A_to_B.parameters(),cycleGAN.G_B_to_A.parameters()) , lr = lr, betas=(0.5, 0.999))
optimizerDA = torch . optim . Adam ( cycleGAN.D_A.parameters(), lr = lr, betas=(0.5, 0.999))
optimizerDB = torch . optim . Adam ( cycleGAN.D_B.parameters(), lr = lr, betas=(0.5, 0.999))
optimizerDAM = torch . optim . Adam ( cycleGAN.D_A_m.parameters(), lr = lr, betas=(0.5, 0.999))
optimizerDBM = torch . optim . Adam ( cycleGAN.D_B_m.parameters() , lr = lr, betas=(0.5, 0.999))

def set_grad(cycleGAN,bool_val):
    for param in cycleGAN.D_A.parameters():
        param.requires_grad = bool_val
    for param in cycleGAN.D_B.parameters():
        param.requires_grad = bool_val
    for param in cycleGAN.D_A_m.parameters():
        param.requires_grad = bool_val
    for param in cycleGAN.D_B_m.parameters():
        param.requires_grad = bool_val


    
# if torch.cuda.is_available():
#     cycleGAN = nn.DataParallel(cycleGAN)
#     cycleGAN = cycleGAN.cuda()



In [ ]:
for epoch in range(epochs):
    
    try:
        cycleGAN = load_model('cycleGAN')
    except:
        print('Load Failed')
        pass

    running_lossD = 0.0
    running_lossG = 0.0
    idxminibatches_A = np.random.permutation(NB)
    idxminibatches_B = np.random.permutation(NB)
    idxminibatches_M = np.random.permutation(NB)
    
    for k in range(NB):
        import gc
        gc.collect()
        
        i_A = idxminibatches_A[k]
        i_B = idxminibatches_B[k]
        i_M = idxminibatches_M[k]
        
        idx_A = np.arange(B*i_A, min(B*(i_A+1), N))
        idx_B = np.arange(B*i_B, min(B*(i_B+1), N))
        idx_M = np.arange(B*i_M, min(B*(i_M+1), N))
        
        inputs_A = x_A[idx_A]
        inputs_B = x_B[idx_B]
        inputs_M = x_M[idx_M]
        
#         if torch.cuda.is_available():
#             inputs_A.data = inputs_A.data.cuda()
#             inputs_B.data = inputs_B.data.cuda()
#             inputs_M.data = inputs_M.data.cuda()

        set_grad(cycleGAN,False)
        optimizerG.zero_grad()
        lossG = cycleGAN.loss_G(10,inputs_A,inputs_B)
        lossG.backward()
        running_lossG += lossG[0]
        optimizerG.step()
        del lossG
        
        
    
        set_grad(cycleGAN,True)
        optimizerDA.zero_grad()
        lossDA = cycleGAN.loss_DA(inputs_A,inputs_B)
        lossDA.backward()
        running_lossD += lossDA[0]
        optimizerDA.step()
        del lossDA

        optimizerDB.zero_grad()
        lossDB = cycleGAN.loss_DB(inputs_A,inputs_B)
        lossDB.backward()
        running_lossD += lossDB[0]
        optimizerDB.step()
        del lossDB

        optimizerDAM.zero_grad()
        lossDAM = cycleGAN.loss_DA_m(1,inputs_A,inputs_B,inputs_M)
        lossDAM.backward()
        running_lossD += lossDAM[0]
        optimizerDAM.step()
        del lossDAM

        optimizerDBM.zero_grad()
        lossDBM = cycleGAN.loss_DB_m(1,inputs_A,inputs_B,inputs_M)
        lossDBM.backward()
        running_lossD += lossDBM[0]
        optimizerDBM.step()
        del lossDBM

        
        torch.cuda.empty_cache()
        # Print statistics
        
        
        if k % 100 == 99:
            print('[%d, %5d] lossD: %.3f lossG: %.3f' %
            (epoch + 1, k + 1, running_lossD / 100, running_lossG / 100))
            running_lossG = 0.0
            running_lossD = 0.0
    
    save_model(cycleGAN)


print('Finished Training')

Load Failed
[1,   100] lossD: 24.063 lossG: 39663.988
[1,   200] lossD: 24.087 lossG: 38739.016
[1,   300] lossD: 23.793 lossG: 39285.316
[1,   400] lossD: 25.212 lossG: 40756.703
[1,   500] lossD: 22.945 lossG: 39214.516
[1,   600] lossD: 22.233 lossG: 39392.504
[1,   700] lossD: 23.235 lossG: 38744.723
[1,   800] lossD: 21.671 lossG: 37098.090
[1,   900] lossD: 22.290 lossG: 38847.816
[1,  1000] lossD: 21.930 lossG: 38173.410
[1,  1100] lossD: 21.130 lossG: 38885.668
[1,  1200] lossD: 20.303 lossG: 38598.492
[1,  1300] lossD: 20.728 lossG: 40148.398
[1,  1400] lossD: 20.778 lossG: 39081.895
[1,  1500] lossD: 20.454 lossG: 38372.094
[1,  1600] lossD: 19.624 lossG: 38374.324
[1,  1700] lossD: 19.551 lossG: 39174.289
[1,  1800] lossD: 18.392 lossG: 38391.910


/opt/conda/lib/python2.7/site-packages/torch/serialization.py:159: UserWarning: Couldn't retrieve source code for container of type Generator_Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python2.7/site-packages/torch/serialization.py:159: UserWarning: Couldn't retrieve source code for container of type BasicBlock. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python2.7/site-packages/torch/serialization.py:159: UserWarning: Couldn't retrieve source code for container of type Discriminator_Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Load Failed
[2,   100] lossD: 19.544 lossG: 39121.836
[2,   200] lossD: 18.902 lossG: 38556.387
[2,   300] lossD: 18.512 lossG: 40226.426
[2,   400] lossD: 19.185 lossG: 39487.211
[2,   500] lossD: 19.373 lossG: 37192.195
[2,   600] lossD: 18.831 lossG: 37989.113
[2,   700] lossD: 17.715 lossG: 37238.023
[2,   800] lossD: 17.520 lossG: 38773.777
[2,   900] lossD: 18.286 lossG: 37514.145
[2,  1000] lossD: 17.514 lossG: 39434.617
[2,  1100] lossD: 17.246 lossG: 40778.727
[2,  1200] lossD: 16.772 lossG: 38204.852
[2,  1300] lossD: 17.301 lossG: 39338.473
[2,  1400] lossD: 16.411 lossG: 38257.613
[2,  1500] lossD: 17.384 lossG: 37727.715
[2,  1600] lossD: 16.624 lossG: 39653.836
[2,  1700] lossD: 17.486 lossG: 38098.730
[2,  1800] lossD: 16.140 lossG: 38959.801
Load Failed
[3,   100] lossD: 15.356 lossG: 40022.801
[3,   200] lossD: 15.543 lossG: 37953.984
[3,   300] lossD: 16.304 lossG: 39755.777
[3,   400] lossD: 15.905 lossG: 39402.531
[3,   500] lossD: 15.899 lossG: 37375.371
[3,   600]

[12,  1300] lossD: 21.445 lossG: 27106.865
[12,  1400] lossD: 23.018 lossG: 27772.529
[12,  1500] lossD: 18.964 lossG: 26829.902
[12,  1600] lossD: 18.283 lossG: 27116.297
[12,  1700] lossD: 20.599 lossG: 25934.609
[12,  1800] lossD: 18.765 lossG: 27655.789
Load Failed
[13,   100] lossD: 17.996 lossG: 26946.086
[13,   200] lossD: 17.777 lossG: 27264.996
[13,   300] lossD: 17.466 lossG: 27620.336
[13,   400] lossD: 20.156 lossG: 26780.537
[13,   500] lossD: 20.123 lossG: 26036.809
[13,   600] lossD: 23.125 lossG: 27163.684
[13,   700] lossD: 20.224 lossG: 26751.889
[13,   800] lossD: 17.656 lossG: 26723.955
[13,   900] lossD: 19.381 lossG: 27587.877
[13,  1000] lossD: 18.360 lossG: 26714.244
[13,  1100] lossD: 19.464 lossG: 26244.299
[13,  1200] lossD: 19.271 lossG: 27004.361
[13,  1300] lossD: 17.042 lossG: 27900.850
[13,  1400] lossD: 19.090 lossG: 26971.100
[13,  1500] lossD: 15.528 lossG: 27690.229
[13,  1600] lossD: 15.141 lossG: 26935.625
[13,  1700] lossD: 16.665 lossG: 27453.754

In [15]:
def save_model(cycleGAN):
    torch.save(cycleGAN.G_A_to_B,'G_A_to_B')
    torch.save(cycleGAN.G_B_to_A,'G_B_to_A')
    torch.save(cycleGAN.D_A,'D_A')
    torch.save(cycleGAN.D_B,'D_B')
    torch.save(cycleGAN.D_A_m,'D_A_m')
    torch.save(cycleGAN.D_B_m,'D_B_m')
    
def load_model(cycleGAN):
    cycleGAN.G_A_to_B = torch.load('G_A_to_B')
    cycleGAN.G_A_to_B = torch.load('G_B_to_A')
    cycleGAN.D_A = torch.load('D_A')
    cycleGAN.D_B = torch.load('D_B')
    cycleGAN.D_B_m = torch.load('D_B_m')
    cycleGAN.D_A_m = torch.load('D_A_m')
        

In [35]:
i = inputs_A.data.cpu().numpy()
j = cycleGAN.G_A_to_B(inputs_A).data.cpu().numpy()
np.save('i.npy',i)
np.save('j.npy',j)

In [14]:
import numpy as np
import pretty_midi

def parse_data(data):
    data_ones = data == True
    data_zeros = data == False
    new_data = np.zeros_like(data, dtype=np.float32)
    new_data[data_ones] = 1.0
    new_data[data_zeros] = 0.0
    return new_data

def to_binary(data, threshold=0.5):
    data_ones = data >= threshold
    data_zeros = data < threshold
    new_data = np.zeros_like(data, dtype=np.float32)
    new_data[data_ones] = 1.0
    new_data[data_zeros] = 0.0
    return new_data

def save_midi(data):
    data_binary = to_binary(data)
    save_midis(data_binary, "./sample_outputs_midi")

def set_piano_roll_to_instrument(piano_roll, instrument, velocity=100, tempo=120.0, beat_resolution=16):
    # Calculate time per pixel
    tpp = 60.0 / tempo / float(beat_resolution)
    threshold = 60.0 / tempo / 4
    phrase_end_time = 60.0 / tempo * 4 * piano_roll.shape[0]
    # Create piano_roll_search that captures note onsets and offsets
    piano_roll = piano_roll.reshape((piano_roll.shape[0] * piano_roll.shape[1], piano_roll.shape[2]))
    piano_roll_diff = np.concatenate((np.zeros((1, 128), dtype=int), piano_roll, np.zeros((1, 128), dtype=int)))
    piano_roll_search = np.diff(piano_roll_diff.astype(int), axis=0)
    # Iterate through all possible(128) pitches

    for note_num in range(128):
        # Search for notes
        start_idx = (piano_roll_search[:, note_num] > 0).nonzero()
        start_time = list(tpp * (start_idx[0].astype(float)))
        # print('start_time:', start_time)
        # print(len(start_time))
        end_idx = (piano_roll_search[:, note_num] < 0).nonzero()
        end_time = list(tpp * (end_idx[0].astype(float)))
        # print('end_time:', end_time)
        # print(len(end_time))
        duration = [pair[1] - pair[0] for pair in zip(start_time, end_time)]
        # print('duration each note:', duration)
        # print(len(duration))

        temp_start_time = [i for i in start_time]
        temp_end_time = [i for i in end_time]

        for i in range(len(start_time)):
            # print(start_time)
            if start_time[i] in temp_start_time and i != len(start_time) - 1:
                # print('i and start_time:', i, start_time[i])
                t = []
                current_idx = temp_start_time.index(start_time[i])
                for j in range(current_idx + 1, len(temp_start_time)):
                    # print(j, temp_start_time[j])
                    if temp_start_time[j] < start_time[i] + threshold and temp_end_time[j] <= start_time[i] + threshold:
                        # print('popped start time:', temp_start_time[j])
                        t.append(j)
                        # print('popped temp_start_time:', t)
                for _ in t:
                    temp_start_time.pop(t[0])
                    temp_end_time.pop(t[0])
                # print('popped temp_start_time:', temp_start_time)

        start_time = temp_start_time
        # print('After checking, start_time:', start_time)
        # print(len(start_time))
        end_time = temp_end_time
        # print('After checking, end_time:', end_time)
        # print(len(end_time))
        duration = [pair[1] - pair[0] for pair in zip(start_time, end_time)]
        # print('After checking, duration each note:', duration)
        # print(len(duration))

        if len(end_time) < len(start_time):
            d = len(start_time) - len(end_time)
            start_time = start_time[:-d]
        # Iterate through all the searched notes
        for idx in range(len(start_time)):
            if duration[idx] >= threshold:
                # Create an Note object with corresponding note number, start time and end time
                note = pretty_midi.Note(velocity=velocity, pitch=note_num, start=start_time[idx], end=end_time[idx])
                # Add the note to the Instrument object
                instrument.notes.append(note)
            else:
                if start_time[idx] + threshold <= phrase_end_time:
                    # Create an Note object with corresponding note number, start time and end time
                    note = pretty_midi.Note(velocity=velocity, pitch=note_num, start=start_time[idx],
                                            end=start_time[idx] + threshold)
                else:
                    # Create an Note object with corresponding note number, start time and end time
                    note = pretty_midi.Note(velocity=velocity, pitch=note_num, start=start_time[idx],
                                            end=phrase_end_time)
                # Add the note to the Instrument object
                instrument.notes.append(note)
    # Sort the notes by their start time
    instrument.notes.sort(key=lambda note: note.start)
    # print(max([i.end for i in instrument.notes]))
    # print('tpp, threshold, phrases_end_time:', tpp, threshold, phrase_end_time)

def save_midis(bars, file_path, tempo=80.0):
    padded_bars = np.concatenate((np.zeros((bars.shape[0], bars.shape[1], 24, bars.shape[3])), bars, np.zeros((bars.shape[0], bars.shape[1], 20, bars.shape[3]))), axis=2)
    pause = np.zeros((bars.shape[0], 64, 128, bars.shape[3]))
    images_with_pause = padded_bars
    images_with_pause = images_with_pause.reshape(-1, 64, padded_bars.shape[2], padded_bars.shape[3])
    images_with_pause_list = []
    for ch_idx in range(padded_bars.shape[3]):
        images_with_pause_list.append(images_with_pause[:, :, :, ch_idx].reshape(images_with_pause.shape[0], images_with_pause.shape[1], images_with_pause.shape[2]))
    write_piano_rolls_to_midi(images_with_pause_list, program_nums=[0], is_drum=[False], filename=file_path, tempo=tempo, beat_resolution=4)

def write_piano_rolls_to_midi(piano_rolls, program_nums=None, is_drum=None, filename='test.mid', velocity=100, tempo=120.0, beat_resolution=24):
    if len(piano_rolls) != len(program_nums) or len(piano_rolls) != len(is_drum):
        print("Error: piano_rolls and program_nums have different sizes...")
        return False
    if not program_nums:
        program_nums = [0, 0, 0]
    if not is_drum:
        is_drum = [False, False, False]
    # Create a PrettyMIDI object
    midi = pretty_midi.PrettyMIDI(initial_tempo=tempo)
    # Iterate through all the input instruments
    for idx in range(len(piano_rolls)):
        # Create an Instrument object
        instrument = pretty_midi.Instrument(program=program_nums[idx], is_drum=is_drum[idx])
        # Set the piano roll to the Instrument object
        set_piano_roll_to_instrument(piano_rolls[idx], instrument, velocity, tempo, beat_resolution)
        # Add the instrument to the PrettyMIDI object
        midi.instruments.append(instrument)
    # Write out the MIDI data
    midi.write(filename)

data1 = inputs_A.data.cpu()

print(data1.shape)
# data1 = to_binary(parse_data(data1))
# save_midi(data1)

ImportError: No module named pretty_midi

In [30]:
import pip


def install(package):
    if hasattr(pip, 'main'):
        pip.main(['install', package])
    else:
        pip._internal.main(['install', package])

pip.main(['install','--upgrade' ,'pip'])
install('pretty-midi')

  Using cached https://files.pythonhosted.org/packages/c2/d7/90f34cb0d83a6c5631cf71dfe64cc1054598c843a92b400e55675cc2ac37/pip-18.1-py2.py3-none-any.whl
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:


Exception:
Traceback (most recent call last):
  File "/opt/conda/lib/python2.7/site-packages/pip/basecommand.py", line 215, in main
    status = self.run(options, args)
  File "/opt/conda/lib/python2.7/site-packages/pip/commands/install.py", line 342, in run
    prefix=options.prefix_path,
  File "/opt/conda/lib/python2.7/site-packages/pip/req/req_set.py", line 778, in install
    requirement.uninstall(auto_confirm=True)
  File "/opt/conda/lib/python2.7/site-packages/pip/req/req_install.py", line 754, in uninstall
    paths_to_remove.remove(auto_confirm)
  File "/opt/conda/lib/python2.7/site-packages/pip/req/req_uninstall.py", line 115, in remove
    renames(path, new_path)
  File "/opt/conda/lib/python2.7/site-packages/pip/utils/__init__.py", line 267, in renames
    shutil.move(old, new)
  File "/opt/conda/lib/python2.7/shutil.py", line 317, in move
    os.unlink(src)
OSError: [Errno 13] Permission denied: '/opt/conda/bin/pip'
You are using pip version 9.0.1, however version 18.1 is 

  Using cached https://files.pythonhosted.org/packages/20/0a/81beb587b1ae832ea6a1901dc7c6faa380e8dd154e0a862f0a9f3d2afab9/mido-1.2.9-py2.py3-none-any.whl


Exception:
Traceback (most recent call last):
  File "/opt/conda/lib/python2.7/site-packages/pip/basecommand.py", line 215, in main
    status = self.run(options, args)
  File "/opt/conda/lib/python2.7/site-packages/pip/commands/install.py", line 342, in run
    prefix=options.prefix_path,
  File "/opt/conda/lib/python2.7/site-packages/pip/req/req_set.py", line 784, in install
    **kwargs
  File "/opt/conda/lib/python2.7/site-packages/pip/req/req_install.py", line 851, in install
    self.move_wheel_files(self.source_dir, root=root, prefix=prefix)
  File "/opt/conda/lib/python2.7/site-packages/pip/req/req_install.py", line 1064, in move_wheel_files
    isolated=self.isolated,
  File "/opt/conda/lib/python2.7/site-packages/pip/wheel.py", line 345, in move_wheel_files
    clobber(source, lib_dir, True)
  File "/opt/conda/lib/python2.7/site-packages/pip/wheel.py", line 316, in clobber
    ensure_dir(destdir)
  File "/opt/conda/lib/python2.7/site-packages/pip/utils/__init__.py", line 83, 

In [13]:
[x[0] for x in cycleGAN.named_parameters() if 'G' not in x[0]]

AttributeError: CycleGAN instance has no attribute 'named_parameters'

In [ ]:
print(list(cycleGAN.named_parameters()))


In [ ]:
 cycleGAN.state_dict()

In [ ]:
 torch.cuda.empty_cache()